In [171]:
import os
import re
import csv
import sys
import json
import random
import workFactory as wf

import MeCab

from tqdm import tqdm

csv.field_size_limit(sys.maxsize)

9223372036854775807

In [164]:
TARGET = 42
SIM_PATH = f'./morpheme/similarity/{TARGET}'
PATH = f'./tsv/morpheme/{TARGET}'
simList = wf.load_tsv_similarity(f'{SIM_PATH}/total_avg.tsv')
simKList = [word['word'] for word in simList][:10]
#simKList.append("妹")
works = wf.load_tsv_dataset(f'{PATH}/test.tsv')

50000it [00:00, 713108.41it/s]


In [165]:
'''
for i, work in enumerate(works):
    if len(work['content']) < 2000:
        print(i, work)
'''
##33
work = works[42]
#print(work)
#print(works[42])

In [166]:
def getPScoreOffset(simKList, content, offset, mecab):
    stageCount = len(content)//offset
    result = []
    for i in range(0, stageCount):
        paragraph = content[i*offset:i*offset+512]
        phrase = mecab.parse(paragraph).replace("\n", "")
        sep = re.split(" ", phrase)
        score = 0
        for word in sep:
            if word in simKList:
                score += 1
        if score != 0:
            result.append({
                'pos': i*offset,
                'end': i*offset+512,
                'score': score,
                'content': phrase
            })
    return result

In [172]:
def getJointListByOffset(content, offset):
    content = content.replace("\n", "\n ")
    sep = re.split(" ", content)
    sepList = [] 
    compList = []
    for i, word in enumerate(sep):
        if sum(len(i) for i in sepList) < offset:
            sepList.append(word)
            if i == len(sep)-1:
                compList.append(" ".join(sepList))
        else:
            compList.append(" ".join(sepList))
            sepList.clear()
            sepList.append(word)
    jointList = []
    for i, _ in enumerate(compList):
        try:
            if offset == 256:
                jointList.append(compList[i]+compList[i+1])
            elif offset == 512:
                jointList.append(compList[i])
        except IndexError:
            _
    return jointList

In [237]:
def getPScoreOffsetWOMECAB(simKList, content, offset, mecab):
    jointList = getJointListByOffset(content, offset)
    result = []
    for joint in jointList:
        paragraph = joint.replace("\n", "[SEP]")
        sep = re.split(" ", paragraph)
        score = 0
        for word in sep:
            if word in simKList:
                score += 1
        if score != 0:
            result.append({
                'score': score,
                'content': " ".join(sep)
            })
    
    resultNum = len(result)
    if resultNum < 10:
        extra = 10 - resultNum
        for i in range(0, extra):
            if len(content) > 512:
                rand = random.randint(0, len(content)-512)
            else:
                rand = 0
            randContent = content[rand:]
            randContent = randContent.replace(" ", "")
            randContent = randContent.replace("\n", "\\")
            randContent = randContent[:512]
            randResult = mecab.parse(randContent)
            randResult = randResult.replace("\n", "")
            randResult = randResult.replace("\\", "[SEP]")
            result.append({
                'score': -1,
                'content': randResult
            })
    return resultNum, result

In [239]:
content = work['content']
#print(len(content))
mecab = MeCab.Tagger("-Owakati")
num, scores = getPScoreOffsetWOMECAB(simKList, content, 256, mecab)
print(num)
print(scores)
#print(sorted(scores, key=lambda x: x['score'], reverse=True))
#print(len(scores[65]['content'].replace(" ","")))

2
[{'score': 1, 'content': 'やっ て き た [SEP] 髪 の 長い もはや 妖怪 みたい な の が 、 ぴちゃぴちゃ と 水 を 垂らし ながら 迫っ て くる シーン [SEP] なかなか だっ た [SEP] 夏希 じゃ 耐え られる わけ が ない [SEP] にやり と し ながら 、 前 を 見る [SEP] ん っ [SEP] 思わ ず 声 が 出 そう に なる の を 必死 に 堪える [SEP] 夏希 と お 兄 ちゃん は 手 を 重ね て い た [SEP] な っ 、 夏希 、 この アマ 、 あたし の お 兄 ちゃん に 手 を [SEP] な ん だ か 積極 的 に なっ てる [SEP] いつ も より も 少し 大人 っ ぼく 色づい た 顔 を し て いる の は メーク の せい だろう か [SEP] 一瞬 、 夏希 が はっと 何 か 閃い た 顔 を し た の が 気 に なる [SEP] 夏希 の 横顔 は 幸せ そう で 恐怖 と は 無縁 の 印象 を あたし に 与え た [SEP] な ん だ か 、 妙に 嫌 な 予感 が し た [SEP] お 兄 ちゃん の こと を より 考え出し た の か 、 中盤 から は 何 やら 小 動物 ぽ さ が 増し て 、 耳元 で ささやきあっ たり 、 さらに 甘え だし て いる [SEP] くそ 、 くそう [SEP] こんな はず じゃ ない の に [SEP] ま 、 まさか 、 夏希 の 奴 … … 天才 的 な 頭脳 を 有し て いる あたし の 脳裏 に は 良く ない 考え が 浮かん だ [SEP] あり 得 ない と は 思う が 、 もし そう なら 夏希 が ここ まで 耐え られ た 説明 も つく [SEP] あたし は 勢い よく ポップコーン を 口 に 入れ だし 、 もはや 本編 など どう で も よく なり 、 この ２人 の 方 に 集中 し かけ て い た [SEP] 作品 は クライマックス に 突入 し た [SEP] 恐怖 心 の 煽り が 最高 点 に 達し 、 お 客 さん の'}, {'score': 1, 'content': 'に 嫌 な 予感 が し た [SE

In [30]:
content = work['content']
content = content.replace('\n', '')
print(len(content))
content = content.replace(' ', '')
print(len(content))
#sep = re.split(" ", content)
scores = getPScoreOffset(simKList, sep, 128)

75088
45986
29103
128


In [ ]:
### note: label needs to be return to 2 digit binary -> 10 01

In [ ]:
content = work['content']
content = content.replace('\n', '')
#print(content)
sep = re.split(" ", content)
#print(sep)

In [ ]:
def getPositionScore(matchWords, sep):
    # define variables
    scores = [{'pos': 0, 'len': 0, 'end': 0, 'score':0}]
    wordCount = 0
    scoreCount = 0
    positionCount = 0
    endCount = 0
    
    # calculate position scores
    while endCount < len(sep):
        if wordCount < 512:
            wordCount += len(sep[endCount])
            if sep[endCount] in matchWords:
                scoreCount += 1 # we can change weight by setting into word score
        elif wordCount == 512:
            wordCount -= len(sep[positionCount])
            if sep[positionCount] in matchWords:
                scoreCount -= 1 # we can change weight by setting into word score
            wordCount += len(sep[endCount])
            if sep[endCount] in matchWords:
                scoreCount += 1 # we can change weight by setting into word score
            positionCount += 1
        elif wordCount > 512:
            wordCount -= len(sep[positionCount])
            if sep[positionCount] in matchWords:
                scoreCount -= 1 # we can change weight by setting into word score
            positionCount += 1
            endCount -= 1
        endCount += 1
        if scoreCount >= scores[0]['score'] and wordCount >= 512 and scoreCount > 0:
            scores.insert(0, {'pos': positionCount, 'len': wordCount, 'end': endCount, 'score': scoreCount})
    return scores

In [ ]:
scores = [{'pos': 0, 'len': 0, 'end': 0, 'score':0}]
matchWordList = ['ロゴス', 'た']
wordCount = 0
scoreCount = 0
positionCount = 0
i = 0
while i < (len(sep)):
    if wordCount < 512:
        wordCount += len(sep[i])
        
        if sep[i] in matchWordList:
            scoreCount += 1
            print('<512 + ', sep[i])
        
    elif wordCount == 512:
        wordCount -= len(sep[positionCount])
        wordCount += len(sep[i])
        
        if sep[positionCount] in matchWordList:
            scoreCount -= 1
            print('=512 - ', sep[positionCount])
            
        if sep[i] in matchWordList:
            scoreCount += 1
            print('=512 + ', sep[i])
        
        positionCount += 1
        
    elif wordCount > 512:
        wordCount -= len(sep[positionCount])
        
        if sep[positionCount] in matchWordList:
            scoreCount -= 1
            print('>512 - ', sep[positionCount])
        
        positionCount += 1
        i -= 1
    i += 1     
    if scoreCount >= scores[0]['score'] and wordCount > 500:
        scores.insert(0, {'pos': positionCount, 'len': wordCount, 'end': i, 'score': scoreCount})
    #print(f'pos: {positionCount}  len: {wordCount}, score: {scoreCount}')
"""
for word in sep:
    if wordCount < 512:
        if word in matchWordList:
            scoreCount += 1
        wordCount += len(word)
        #print(word)
        #print('512 line -----')
    else:
        #print(sep[positionCount])  対となる
        #print(word)
        wordCount -= len(sep[positionCount])
        wordCount += len(word)
        if sep[positionCount] in matchWordList:
            scoreCount -= 1
        if word in matchWordList:
            scoreCount += 1
        positionCount += 1
    if scoreCount >= scores[0]['score']:
        scores.insert(0, {'pos': positionCount, 'score': scoreCount})
    print(f'pos: {positionCount}  len: {wordCount}, score: {scoreCount}')
"""
print(scores)


In [ ]:
print(len(''.join(sep[39:365])))

In [ ]:
print(getPositionScore(['ロゴス', 'た'], work))

In [ ]:
simList = wf.load_tsv_similarity(f'{SIM_PATH}/total_avg.tsv')
print(simList[:10])

In [ ]:
simKList = [word['word'] for word in simList][:10]
print(simKList)

In [ ]:
for work in works[:5]:
    score = getPositionScore(simKList, work)[0]
    if score['score'] >= 10:
        print(score)
        content = work['content']
        content = content.replace('\n', '')
        sep = re.split(" ", content)
        print(work['label'])
        #print(' '.join(sep[score['pos']:score['end']]))
        print('--------------------')

In [ ]:
def savePositionScoreTSV(works, simKList, savePath):
    results = []
    for work in tqdm(works[:10]): # TEST
        print(work['label'])
        if work['label'] == 0:
            label = '01'
        else:
            label = '10'
            
        content = work['content']
        content = content.replace('\n', '')
        sep = re.split(" ", content)
    
        score = getPositionScore(simKList, sep)[0]
        
        if score['score'] > 0:
            paragraph = ' '.join(sep[score['pos']:score['end']])
        else:
            paragraph = content[:512]
        result = {'label': label, 'content': paragraph, 'score': score}
        print(result)
        results.append(result)
    
    with open(savePath, 'w+', encoding='utf-8') as f:
        writer = csv.writer(f, delimiter='\t')
        for result in results:
            writer.writerow([result['label'], result['content']])
    
    with open(f'{savePath}_status.tsv', 'a+', encoding='utf-8') as f2:
        writer = csv.writer(f2, delimiter='\t')
        for result in results:
            writer.writerow([result['score']])
            

In [ ]:
def nearestK(target, kSize):
    SIM_PATH = f'./morpheme/similarity/{target}'
    WORK_PATH = f'./tsv/morpheme/{target}'
    SAVE_PATH = f'./tsv/nearest-k/k-{kSize}/{target}'

    trains = wf.load_tsv_dataset(f'{WORK_PATH}/train.tsv')
    tests = wf.load_tsv_dataset(f'{WORK_PATH}/test.tsv')
    devs = wf.load_tsv_dataset(f'{WORK_PATH}/dev.tsv')
    
    simList = wf.load_tsv_similarity(f'{SIM_PATH}/total_avg.tsv')
    simKList = [word['word'] for word in simList][:kSize]
    
    os.makedirs(SAVE_PATH, exist_ok=True)
    savePositionScoreTSV(trains, simKList, f'{SAVE_PATH}/train.tsv')
    savePositionScoreTSV(tests, simKList, f'{SAVE_PATH}/test.tsv')
    savePositionScoreTSV(devs, simKList, f'{SAVE_PATH}/dev.tsv')

In [ ]:
nearestK(42, 10)

In [ ]:
TARGET = 42
SIM_PATH = f'./morpheme/similarity/{TARGET}'
PATH = f'./tsv/morpheme/{TARGET}'

works = wf.load_tsv_dataset(f'{PATH}/test.tsv')

In [ ]:
work = works[3]
content = work['content']
content = content.replace('\n', '')
sep = re.split(" ", content)
#print(sep)
simList = wf.load_tsv_similarity(f'{SIM_PATH}/total_avg.tsv')
simKList = [word['word'] for word in simList][:100]
print(simKList)
scores = getPositionScore(simKList, sep)

In [ ]:
candidates = [scores[0]]
#highScore = scores[0]['score']
for score in scores:
    #print(f'{score}\n{candidates[0]}\n------------')
    if score['end'] < candidates[0]['pos'] and score['score'] >= 0:
        candidates.insert(0, score)
        
if candidates[0]['score'] == 0:
    candidates.pop(0)

candidateList = []
if len(candidates)!= 0:
    for candidate in candidates:
        print(candidate)
        candidateList.append(' '.join(sep[candidate['pos']:candidate['end']]))
else:
    candidateList.append(content[:512])
    
for l in candidateList:
    print(l)

In [ ]:
def getNearestKCandidates(works, simKList, savePath):
    for i, work in enumerate(works[:10]):
        content = work['content']
        content = content.replace('\n', '')
        sep = re.split(" ", content)
        scores = getPositionScore(simKList, sep)
        
        candidates = [scores[0]]
        for score in scores:
            if score['end'] < candidates[0]['pos'] and score['score'] >= 0:
                candidates.insert(0, score)
        
        if candidates[0]['score'] == 0:
            candidates.pop(0)

        candidateList = []
        if len(candidates)!= 0:
            for candidate in candidates:
                print(candidate)
                candidateList.append({
                    'pos': candidate['pos'],
                    'len': candidate['len'],
                    'end': candidate['end'],
                    'score': candidate['score'],
                    'content': ' '.join(sep[candidate['pos']:candidate['end']])
                })
        else:
            candidateList.append({
                'pos': 0,
                'len': 512,
                'end': 512,
                'score': 0,
                'content': content[:512]
            })
        
        if work['label'] == 0:
            label = '01'
        else:
            label = '10'
        
        writeTarget = {
            'label': label,
            'status': candidateList[slice(None, None, -1)]
        }
    
        with open(f'{savePath}/{i}.json', 'w+', encoding='utf-8') as f:
            json.dump(writeTarget, f)

In [ ]:
target = 42
kSize = 100

SIM_PATH = f'./morpheme/similarity/{target}'
WORK_PATH = f'./tsv/morpheme/{target}'
SAVE_PATH = f'./tsv/n-test/n-{kSize}/{target}'

#trains = wf.load_tsv_dataset(f'{WORK_PATH}/train.tsv')
tests = wf.load_tsv_dataset(f'{WORK_PATH}/test.tsv')
#devs = wf.load_tsv_dataset(f'{WORK_PATH}/dev.tsv')

simList = wf.load_tsv_similarity(f'{SIM_PATH}/total_avg.tsv')
simKList = [word['word'] for word in simList][:kSize]

os.makedirs(f'{SAVE_PATH}/test', exist_ok=True)
#getNearestKCandidates(trains, simKList, f'{SAVE_PATH}/train.tsv')
getNearestKCandidates(tests, simKList, f'{SAVE_PATH}/test')
#getNearestKCandidates(devs, simKList, f'{SAVE_PATH}/dev.tsv')